# NOTE
Con Outlier (con e senza Standardizzazione): -> La regressione lineare è invariante rispetto a trasformazioni lineari delle feature
- MSE: 0.557
- R^2: 0.575

Senza Outlier (con e senza Standardizzazione):
- MSE: 0.361
- R^2: 0.588

# IMPORT LIBRERIE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# CARICAMENTO DATASET

In [ ]:
data = pd.read_csv("california_housing_data.csv")
data.head()

In [ ]:
# Info dataset
data.info()

# Statistiche descrittive
data.describe()

Ho tutti numeri (niente stringhe) non nulli -> niente pulizia

## Outlier

In [ ]:
# Calcolo Q1 e Q3
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Mantengo solo i dati senza outlier
data_clean = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]

## Separazione variabili

In [ ]:
# X (features/caratteristiche) -> tutte le colonne che userai per fare la predizione
# y (target) -> la colonna che vuoi predire, cioè il prezzo della casa

X = data_clean.drop(["MedHouseVal", "Population", "AveOccup"], axis=1) # caratteristiche -> prendo tutte le colonne tranne quelle indicate
y = data_clean["MedHouseVal"] # target

## Standardizza le caratteristiche utilizzando StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Visualizzazione della standardizzazione

In [ ]:
# Converto in DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

for col in X.columns:
    plt.figure(figsize=(10, 4))
    
    plt.plot(X.index, X[col], label=f"{col} (originale)", alpha=0.6)
    plt.plot(X_scaled_df.index, X_scaled_df[col], label=f"{col} (scalato)", linestyle='--', alpha=0.8)
    
    plt.title(f"Standardizzazione della feature: {col}")
    plt.xlabel("Indice campione")
    plt.ylabel("Valore")
    
    plt.legend()
    plt.tight_layout()
    plt.show()

# TRAINING

## Divisione train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Modello di regressione lineare

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train) # Calcola i pesi per minimizzare l’errore quadratico medio sui dati di training

## Predizione

In [ ]:
y_pred = model.predict(X_test)

## Valutazione performance

- MSE (Mean Squared Error) -> più basso è meglio
- R² -> indica quanto il modello spiega la varianza dei dati (1 = perfetto, 0 = niente relazione)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.3f}")
print(f"R^2: {r2:.3f}")

### Visualizzare predizioni vs valori reali

In [ ]:
plt.figure(figsize=(6,6))

plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # linea y=x

plt.xlabel("Valori reali")
plt.ylabel("Predizioni")
plt.title("Predizioni vs Valori reali")
plt.show()